In [2]:
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5
from datetime import datetime as DT
import hmac
from Crypto.Cipher import AES
from merklelib import MerkleTree
import warnings
warnings.filterwarnings("ignore")

In [3]:
class device():
    deviceCount = 0
    
    def __init__(self):
        self.id = device.deviceCount
        self.keyPair,self.pk,self.__sk= self.getKeys()
        
        self.idProof = {}
        self.idProof['public_KEY'] = self.pk
        self.idProof['block_no'] = None
        self.idProof['block_hash'] = None
        self.idProof['difficulty'] = None
        self.idProof['txn_Index'] = None
        self.idProof['Node_ID'] = None
        self.idProof['okToSend'] = None
        self.idProof['okToRecieve'] = None
        
        data=str(self.id)+self.pk
        self.address=SHA.new(data.encode()).hexdigest()
        device.deviceCount += 1
    
    def getKeys(self):
        keypair=RSA.generate(1024)
        sk = keypair.exportKey('DER')
        pk = keypair.publickey().exportKey('DER')
        return keypair,pk.hex(),sk.hex()
    
    def getDeviceDetails(self):
        print("deviceID: "+str(self.id),"publicKEY:",self.pk,"deviceADDRESS:",self.address,sep="\n",end="\n\n")
        
    @classmethod
    def transact(self, S,R,amt):
        '''
        S: Sender object
        R: Receiver object
        amt: Amount to transfer
        
        if verified, returns a list of dictionaries; dict: {"Sender":S,"Receiver":R,"Amount":amt}
        or else returns []
        '''
        Txn= S.address+R.address+amt
        
        sign,verified = device.signVerify(Txn,S.keyPair)
        if(verified):
            print("Transaction is verified")
            return [{"Sender":S,"Receiver":R,"Amount":amt,"sign":sign}]
        else:
            invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"TxnNotVerified"})
            print("Fraudulent transaction")
            return []
    
    @classmethod
    def dataTransfer(self,S,R,msg,dtKEY):
        """
        Returns [ verified, cipherText, success/failure ]
        """
        if(len(msg)%16!=0):
            pad=16-len(msg)%16
            msg=msg.zfill(pad+len(msg))
        
        sF,rF=device.checkID(S,R)
        secret=AES.new(dtKEY)
        cipherTxt=secret.encrypt(msg).hex()
        
        if(not sF):
            print("Sender ID is not verified")
            return [sF,cipherTxt,"SenderNotVerified"]
        if(not rF):
            print("Receiver ID is not verified")
            return [rF,cipherTxt,"ReceiverNotVerified"]
        
        sign,verified=device.signVerify(cipherTxt,S.keyPair)
        
        if(verified):
            return [verified,cipherTxt,sign]
        else:
            return [verified,cipherTxt,"Fraudulent transaction"]
        
    @classmethod
    def checkID(self, S, R):
        SID=S.idProof
        RID=R.idProof
        sF,rF=True,True    #F-> flag
        
        if(None in SID.values()):
            sF=False
        if(None in RID.values()):
            rF=False
        
        return (sF,rF)
        
    
    @classmethod
    def signVerify(self,txn, sk):
        txn=str(txn)
        msg=SHA.new(txn.encode())
        author=PKCS1_v1_5.new(sk)
        signature = author.sign(msg)
        verified=author.verify(msg,signature)
        return signature,verified
    

    
    def __del__(self):
        print("Device:",self.id,"deleted")
        device.deviceCount-=1

In [4]:
class block:
    blockCount=0
    
    def __init__(self,pH,txnList,PoW,mR):
        self.blockNo=block.blockCount
        block.blockCount+=1
        
        self.prevHash=pH
        self.transactions=txnList
        self.timestamp=DT.timestamp(DT.now())
        self.hash=PoW['Hash']
        self.nonce=PoW['nonce']
        self.mRoot=mR
    
    def getBlockDetails(self):
        print("Block#:",self.blockNo)
        print("previous HASH:",self.prevHash)
        print("current HASH:",self.hash)
        print()
        
        print("Transactions:",len(self.transactions),"     Time:",DT.fromtimestamp(self.timestamp))
        for T in self.transactions:
            print("Sender:",T['Sender'].id, end=",  ")
            print("Receiver:",T['Receiver'].id, end=",  ")
            print("Amount:",T['Amount'])
        
        print("Merkley Root:",self.mRoot)
        print("\nNONCE:",self.nonce)

In [5]:
def HF(data):
    data=data.hex()
    print(data)
    return SHA.new(data.encode()).hexdigest()

In [6]:
class blockChain:
    
    def __init__(self,H):
        self.chain=[]
        self.allTxns={}
        self.Tcount=1

        pH='0'
        PoW={}
        txn=[{"Sender":H,"Receiver":H,"Amount":0}]
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txn)
        mR=SHA.new(str(txn).encode()).hexdigest()
        txID = str(1)
        diff=3
        H.idProof['block_no'] = len(self.chain)
        H.idProof['block_hash'] = PoW['Hash']
        H.idProof['difficulty'] = diff
        H.idProof['txn_Index'] = txID
        pk=H.pk.encode()
        key=hmac.HMAC(pk)
        H.idProof['Node_ID'] = hmac.HMAC(key.digest(),txID.encode()).hexdigest()
        H.idProof['okToSend'] = '0'
        H.idProof['okToRecieve'] = '0'
        
        BLOC=block(pH,txn,PoW,mR)
        self.chain.append(BLOC)
        
        
    def newBlock(self,txn):
        if(txn==[]):
            return
        
        Tno="Txn-"+str(self.Tcount)
        self.allTxns[Tno]=txn[0]
        self.Tcount+=1
        pH=self.chain[-1].hash
        PoW={}
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txn)
        mT=MerkleTree(txn, HF)
        mR=mT.merkle_root
        #ID PROOF GENERATION
        S = txn[0]['Sender']
        txID = str(1)
        diff=3
        S.idProof['block_no'] = len(self.chain)
        S.idProof['block_hash'] = PoW['Hash']
        S.idProof['difficulty'] = diff
        S.idProof['txn_Index'] = txID
        pk=S.pk.encode()
        key=hmac.HMAC(pk)
        S.idProof['Node_ID'] = hmac.HMAC(key.digest(),txID.encode()).hexdigest()
        S.idProof['okToSend'] = '0'
        S.idProof['okToRecieve'] = '0'
        
        BLOC=block(pH,txn,PoW,mR)
        self.chain.append(BLOC)
        
    def addNewBlock(self,txnList):
        if(txnList==[]):
            return
        
        for T in txnList:
            Tno="Txn-"+str(self.Tcount)
            self.allTxns[Tno]=T
            self.Tcount+=1
            
        pH=self.chain[-1].hash
        PoW={}
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txnList)        
        mT = MerkleTree(txnList)
        mR=mT.merkle_root
        BLOC=block(pH,txnList,PoW,mR)
        self.chain.append(BLOC)
        
    def PoW(self,pH,txn,diff=4):
        nonce=0
        data=pH+str(txn)+str(nonce)
        nH=SHA.new(data.encode()).hexdigest()
        while nH[:diff] != '0'*diff:
            nonce += 1
            data=pH+str(txn)+str(nonce)
            nH=SHA.new(data.encode()).hexdigest()
        return nH,nonce
    
    def getChainDetails(self):
        for D in self.chain:
            D.getBlockDetails()
            print("----------------------------------------------------")

In [7]:
dtKEY="sixteen--SIXTEEN"
N=int(input("Enter no.of IoT devices:"))


dList=[device() for i in range(N)]

show=input("press \'Y' to show device details, any other key to skip:")
if(show in ['Y','y']):
    for i in dList:
        i.getDeviceDetails()

#BLOCKCHAIN creati0n
myChain=blockChain(dList[0])
# myChain.getChainDetails()
for i in range(1,N):
    Txn = input("Sender#:   Receiver#:   Amount:").strip().split()
    S,R = [int(x) for x in Txn[:2]]
    amt=Txn[-1]
    if(0<S<N and R==0):
        oneTxnList=device.transact(dList[S],dList[R],amt)
        myChain.newBlock(oneTxnList)
        continue
    if(S>=N):
        S,R,error=str(S),str(R),"SenderNotFound"
        print("Sender not found")
    elif(S==0):
        S,R,error=dList[S],str(R),"invalidSender"
        print("Invalid Sender")
    elif(R != 0 and R<N):
        S,R,error=dList[S],dList[R],"invalid Receiver"
        print("invalid Receiver")
    else:
        S,R,error=dList[S],str(R),"ReceiverNotFound"
        print("Receiver not found")
    invalidTxns.append({"Sender":S,"Receiver":R,"Amount":amt,"status":error})

Enter no.of IoT devices:3
press 'Y' to show device details, any other key to skip:3
Sender#:   Receiver#:   Amount:1 0 0
Transaction is verified
007b2753656e646572273a203c5f5f6d61696e5f5f2e646576696365206f626a656374206174203078303030303032353038453935453945383e2c20275265636569766572273a203c5f5f6d61696e5f5f2e646576696365206f626a656374206174203078303030303032353038453934353234303e2c2027416d6f756e74273a202730272c20277369676e273a2062272c5c7864665c7839655c7861662a575c786634366a5c7863625c7863345c7830385c7861345c7839655c7831345c7830652f735c7865623b375c7839635c786634375c7861305c7830384a5c7866625c7861655c786633315c786264565c786662395c7863625c7862645c7863335c7830375c7865335c786364325c7865655c7862384e457c4d5c7839357a4f5c7866345c7838355c7864325c7839325c78383964285c7864342b295c7862365c786465585c7831635c7861395c7839615c7864642f5c7830325c5c634d5a5c7862395c786634225c7830315c783937287b22565b5c7862315c7865345c7838365c7839645c783931705c783161425c7838302f5c783134587826587a325c7838376d65295c7865385c7862667

In [8]:
invalidTxns=[]

### Displaying each block

In [9]:
myChain.getChainDetails()

Block#: 0
previous HASH: 0
current HASH: 0000b7d691ee2b111d1b233f6e677c46938031ad

Transactions: 1      Time: 2019-12-02 16:03:10.895364
Sender: 0,  Receiver: 0,  Amount: 0
Merkley Root: 7824a688fcd700968442056867e6082c6eb397b5

NONCE: 181389
----------------------------------------------------
Block#: 1
previous HASH: 0000b7d691ee2b111d1b233f6e677c46938031ad
current HASH: 00003a15e0bb60f6202c797697b055419bc24c22

Transactions: 1      Time: 2019-12-02 16:03:13.271704
Sender: 1,  Receiver: 0,  Amount: 0
Merkley Root: 57a87074178f47b130ec1c639a11db2a8a369d12

NONCE: 86538
----------------------------------------------------
Block#: 2
previous HASH: 00003a15e0bb60f6202c797697b055419bc24c22
current HASH: 00003d8cbb74f7525c8d35d6d3cc0d66e9ae926f

Transactions: 1      Time: 2019-12-02 16:03:14.642612
Sender: 2,  Receiver: 0,  Amount: 0
Merkley Root: 3bd647ab7ecd0f6a232307bd7d94a2dcfea48c59

NONCE: 16807
----------------------------------------------------


### All transactions

In [10]:
for k,v in myChain.allTxns.items():
    print(k,": Sender:",v["Sender"].address[:20], "   Receiver:",v["Receiver"].address[:20],   "   Amount:",v['Amount'],  "   Sign:",v["sign"].hex()[:15]   )        

Txn-1 : Sender: 3fd1c72fbc242f86d457    Receiver: 3269f5c61949f35e7f69    Amount: 0    Sign: 2cdf9eaf2a57f43
Txn-2 : Sender: 3dc3261eb437d426f061    Receiver: 3269f5c61949f35e7f69    Amount: 0    Sign: 245edd043c73571


### Displaying ID Proof

In [11]:
for D in dList:
    print("Device:",D.id)
    for k,v in D.idProof.items():
        print(k+":",v)
    print()

Device: 0
public_KEY: 30819f300d06092a864886f70d010101050003818d0030818902818100b1db7040271602221baa078b6639e4e074fb67dfc4efe841b1ea7e8193d7d818523647b1246e386497267921e48a7045f3b21506736299db807bb8d068d84abaeae1f0980009c1e0ce3eb79ec6299f46b3b8d74c72e7614de27b90610c047d577d2c3c7ef282e69df3483d9d4d4dbc3f0329395a5a0de06574cdea0651aeb18d0203010001
block_no: 0
block_hash: 0000b7d691ee2b111d1b233f6e677c46938031ad
difficulty: 3
txn_Index: 1
Node_ID: b1c633cf1c1324bfa223346b564500e8
okToSend: 0
okToRecieve: 0

Device: 1
public_KEY: 30819f300d06092a864886f70d010101050003818d0030818902818100a278bbf035eb775fd623dfdef71244e4e9d9a76e767df7e2ff7cf0f63a554cd7788eee68963d2bbe82ddfa1b0299e74641135323b6e0e8b29e12d71d9d6b91e59322bc4f670a0cc3f241ea89b521e66612e8471c69c277c34b5b6593241dd67b3434563fbb05c8fb9fad33774b2240d174215a840feec7e5ca482a4cdb6c60290203010001
block_no: 1
block_hash: 00003a15e0bb60f6202c797697b055419bc24c22
difficulty: 3
txn_Index: 1
Node_ID: 05759375997cb79f86eaf2f5be754fbe
okToSend: 

## Data Transfer

In [12]:
def performTransactions(bc,dList,I):
    V=[]
    while(1):
        cmd=input("Press N to send new data, any other key to stop sending:")
        if(cmd in ['N','n']):

            S=int(input("Enter Sender's id:"))
            R=int(input("Enter Receiver's id:"))
            msg=input("Enter data to be sent:")

            if(S>=len(dList) or S<0):
                I.append({"Sender":str(S),"Receiver":str(R),"Amount":msg,"status":"inValidSender"})
                print("invalid sender")
                continue
            if(R>=len(dList) or R<0):
                I.append({"Sender":str(S),"Receiver":str(R),"Amount":msg,"status":"inValidReceiver"})
                print("invalid Receiver")
                continue

            data=device.dataTransfer(dList[S],dList[R],msg,dtKEY)
            if(data[0]):
                V.append({"Sender":dList[S],"Receiver":dList[R],"Amount":data[1],"sign":data[2]})
                print("SUCCESSFUL\n")
            else:
                I.append({"Sender":dList[S],"Receiver":dList[R],"Amount":data[1],"status":data[2]})
                print(data[2])

            if(len(V)==3):
                bc.addNewBlock(V)
                V=[]
        else:
            if(bool(V)):
                bc.addNewBlock(V)
                V=[]
            break

In [13]:
performTransactions(myChain,dList,invalidTxns)

Press N to send new data, any other key to stop sending:n
Enter Sender's id:1
Enter Receiver's id:2
Enter data to be sent:fthfh
SUCCESSFUL

Press N to send new data, any other key to stop sending:k


In [14]:
myChain.getChainDetails()

Block#: 0
previous HASH: 0
current HASH: 0000b7d691ee2b111d1b233f6e677c46938031ad

Transactions: 1      Time: 2019-12-02 16:03:10.895364
Sender: 0,  Receiver: 0,  Amount: 0
Merkley Root: 7824a688fcd700968442056867e6082c6eb397b5

NONCE: 181389
----------------------------------------------------
Block#: 1
previous HASH: 0000b7d691ee2b111d1b233f6e677c46938031ad
current HASH: 00003a15e0bb60f6202c797697b055419bc24c22

Transactions: 1      Time: 2019-12-02 16:03:13.271704
Sender: 1,  Receiver: 0,  Amount: 0
Merkley Root: 57a87074178f47b130ec1c639a11db2a8a369d12

NONCE: 86538
----------------------------------------------------
Block#: 2
previous HASH: 00003a15e0bb60f6202c797697b055419bc24c22
current HASH: 00003d8cbb74f7525c8d35d6d3cc0d66e9ae926f

Transactions: 1      Time: 2019-12-02 16:03:14.642612
Sender: 2,  Receiver: 0,  Amount: 0
Merkley Root: 3bd647ab7ecd0f6a232307bd7d94a2dcfea48c59

NONCE: 16807
----------------------------------------------------
Block#: 3
previous HASH: 00003d8cbb7

### All valid txns

In [15]:
for k,v in myChain.allTxns.items():
    print(k,": Sender:",v["Sender"].address[:15], "   Receiver:",v["Receiver"].address[:15],  "   Sign:",v["sign"].hex()[:15],   "   Amount:",v['Amount'][:15]   )        

Txn-1 : Sender: 3fd1c72fbc242f8    Receiver: 3269f5c61949f35    Sign: 2cdf9eaf2a57f43    Amount: 0
Txn-2 : Sender: 3dc3261eb437d42    Receiver: 3269f5c61949f35    Sign: 245edd043c73571    Amount: 0
Txn-3 : Sender: 3fd1c72fbc242f8    Receiver: 3dc3261eb437d42    Sign: 76079dc0a1393dc    Amount: ef887966eee3c28


In [15]:
intruder=device()

sender=dList[0]   #Valid sender

data=device.dataTransfer(sender,intruder,msg="switchOnTheLight",dtKEY=dtKEY)

Receiver ID is not verified


In [32]:
print(data)

if(data[0]):
    txnList.append({"Sender":sender,"Receiver":intruder,"Amount":data[1]})
    print("SUCCESSFUL\n")
else:
    invalidTxns.append({"Sender":sender,"Receiver":intruder,"Amount":data[1],"status":data[2]})

[False, '6f7e7da4d2048507390941e3ba829e56', 'ReceiverNotVerified']


In [33]:
for T in invalidTxns:
    try:
#         print(k,":",v.address[:15],end="  ")
        print("Sender:",T["Sender"].address[:20],   "  Receiver:",T["Receiver"].address[:20],
              "  Amount:",T["Amount"][:20],    "  status:",T["status"])
    except:
        print("Sender:",T["Sender"],   "  Receiver:",T["Receiver"],
              "  Amount:",T["Amount"],    "  status:",T["status"])

Sender: 02ed843201b28583a1f1   Receiver: ba53f383fe5430ebf9a1   Amount: 6f7e7da4d20485073909   status: ReceiverNotVerified
Sender: 02ed843201b28583a1f1   Receiver: ba53f383fe5430ebf9a1   Amount: 6f7e7da4d20485073909   status: ReceiverNotVerified
Sender: 02ed843201b28583a1f1   Receiver: ba53f383fe5430ebf9a1   Amount: 6f7e7da4d20485073909   status: ReceiverNotVerified
